In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8

In [2]:
!pip install torchmetrics evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.7 MB/s eta 0:00:00


In [3]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/100k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.52G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

In [4]:
from torchmetrics.text import BLEUScore
import evaluate

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 1782,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2025.4.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

def formatting_promts_func(examples):
  convos = examples["conversations"]
  texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
  return {"text": texts,}

In [7]:
from datasets import load_dataset

dataset = load_dataset("Mykes/rus_med_dialogues")

README.md:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.96M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/340k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3014 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/335 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['topic', 'user_question', 'assistant_answer', 'to_doctor', '__index_level_0__', 'prompt'],
        num_rows: 3014
    })
    test: Dataset({
        features: ['topic', 'user_question', 'assistant_answer', 'to_doctor', '__index_level_0__', 'prompt'],
        num_rows: 335
    })
})

In [ ]:
import pandas as pd
df = pd.DataFrame(dataset["train"])
df.head()

,topic,user_question,assistant_answer,to_doctor,__index_level_0__,prompt
0,кардиология,У меня учащенное сердцебиение после физической...,Учащенное сердцебиение после физической нагруз...,кардиолог,804,<s><|user|>У меня учащенное сердцебиение после...
1,терапия,"Я принимаю антибиотик Амоксициллин уже 5 дней,...","Понимаю, что кашель не проходит, несмотря на п...",терапевт,501,<s><|user|>Я принимаю антибиотик Амоксициллин ...
2,эндокринология,"У меня постоянно скачет давление. Врач сказал,...",По скачкам давления у Вас может быть действите...,эндокринолог,1448,<s><|user|>У меня постоянно скачет давление. В...
3,травматология,У меня вот уже неделю болит колено после паден...,"Из-за боли в колене после падения, я рекоменду...",травматолог,725,<s><|user|>У меня вот уже неделю болит колено ...
4,онкология,"У меня обнаружили атипию клеток в мазке, что э...",Атипия клеток в мазке может быть признаком раз...,онколог,781,<s><|user|>У меня обнаружили атипию клеток в м...


In [8]:
dataset = dataset.remove_columns(["topic", "to_doctor", "__index_level_0__", "prompt"])

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['user_question', 'assistant_answer'],
        num_rows: 3014
    })
    test: Dataset({
        features: ['user_question', 'assistant_answer'],
        num_rows: 335
    })
})

In [10]:
def standartize(row):
  row["conversations"] = [{'role': 'user', 'content': row['user_question']}, {'role': 'assistant', 'content': row['assistant_answer']}]
  return row

In [11]:
dataset = dataset.map(standartize)

Map:   0%|          | 0/3014 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

In [14]:
dataset = dataset.remove_columns(["user_question", "assistant_answer"])

In [15]:
dataset = dataset.map(formatting_promts_func, batched = True)

Map:   0%|          | 0/3014 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

In [17]:
dataset['train'][5]

{'conversations': [{'content': 'Я заметил, что после употребления молочных продуктов у меня начинают болеть суставы. Могу ли я вылечить эту проблему с помощью диеты?',
   'role': 'user'},
  {'content': 'Возможно, у Вас имеется непереносимость лактозы, которая проявляется болями в суставах после употребления молочных продуктов. Для подтверждения диагноза рекомендуется обратиться к гастроэнтерологу. Врач может назначить Вам тесты на выявление непереносимости лактозы. Если диагноз подтвердится, Вам необходимо исключить или ограничить потребление молочных продуктов из рациона. Существуют специальные лактозой-безопасные молочные продукты, которые содержат фермент лактазу, разлагающий лактозу.  Также можно попробовать заменить обычные молочные продукты на растительные аналоги, например, соевое молоко или миндальное молоко. В любом случае, важно проконсультироваться с врачом или диетологом для составления индивидуального плана питания, который учтет Ваши особенности и потребности.',
   'role'

In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3014 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/335 [00:00<?, ? examples/s]

In [19]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part="<|im_start|>assistant",
)

Map (num_proc=2):   0%|          | 0/3014 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/335 [00:00<?, ? examples/s]

In [20]:
tokenizer.decode(trainer.train_dataset[6]["input_ids"])

'<｜begin▁of▁sentence｜><|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nУ меня последние пару недель болит голова затылочной части.  Боли бывают как во время, так и после напряжения.  Что это может быть?<|im_end|>\n<|im_start|>assistant\nГоловные боли затылочной части могут быть вызваны многими причинами, от напряжения до более серьезных заболеваний.  Важно обратиться к врачу-неврологу для постановки точного диагноза.  Врач проведет  обследование и, возможно, назначит дополнительные исследования.  Возможно, Вам помогут обезболивающие препараты, а также рекомендации по изменению образа жизни, например,  уменьшение стресса и  ежедневным физическим нагрузкам.  Не занимайтесь самолечением, так как это может ухудшить ваше состояние.<|im_end|>\n'

In [21]:
space = tokenizer(" ", add_special_tokens=False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[6]["labels"]])

'                                                                                           \nГоловные боли затылочной части могут быть вызваны многими причинами, от напряжения до более серьезных заболеваний.  Важно обратиться к врачу-неврологу для постановки точного диагноза.  Врач проведет  обследование и, возможно, назначит дополнительные исследования.  Возможно, Вам помогут обезболивающие препараты, а также рекомендации по изменению образа жизни, например,  уменьшение стресса и  ежедневным физическим нагрузкам.  Не занимайтесь самолечением, так как это может ухудшить ваше состояние.<|im_end|>\n'

In [22]:
eval_ds = dataset["test"].select(range(100))

In [23]:
FastLanguageModel.for_inference(model)

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": "У меня учащенное сердцебиение после физической нагрузки. Это нормально?"}],
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

attention_mask = (inputs != tokenizer.pad_token_id).int().to("cuda")

outputs = model.generate(input_ids=inputs, max_new_tokens = 64, use_cache = True, temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nУ меня учащенное сердцебиение после физической нагрузки. Это нормально?<|im_end|>\n<|im_start|>assistant\nПривет! 😊 У بك physical нагрузки может повлиять на сердцебиение, но все же важно отслеживать свои健康- Indicators. Если есть дополнительные вопросы, посмотрим, можно помочь.\n<|im_end|>\n\n</think>\n\nПривет'

In [25]:
from tqdm import tqdm

In [26]:
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [28]:
bleu = BLEUScore(n_gram=1)
sum_init_bleu_score = 0.
sum_init_meteor_score = 0.

for row in tqdm(eval_ds):
  FastLanguageModel.for_inference(model)
  messages = [row['conversations'][0]]

  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True,
      return_tensors = "pt",
  ).to("cuda")

  outputs = model.generate(input_ids=inputs, max_new_tokens = 64, use_cache = True, temperature = 1.5, min_p = 0.1)
  pred = tokenizer.batch_decode(outputs)[0]
  target = row["text"]
  row["bleu"] = bleu(target, pred)

  metric_bleu = bleu(target, pred)
  sum_init_bleu_score += metric_bleu.item()
  metric_meteor = meteor.compute(predictions=[pred], references=[target])
  sum_init_meteor_score += metric_meteor['meteor']

  print("\navg bleu:", sum_init_bleu_score * 1. / len(eval_ds))
  print("\navg meteor:", sum_init_meteor_score * 1. / len(eval_ds))

  1%|          | 1/100 [00:07<13:08,  7.96s/it]


avg bleu: 0.006616641879081726

avg meteor: 0.003555425961024445


  2%|▏         | 2/100 [00:12<09:35,  5.87s/it]


avg bleu: 0.012630611658096313

avg meteor: 0.008592941198252253


  3%|▎         | 3/100 [00:15<07:31,  4.65s/it]

...


avg bleu: 0.5665475532412529

avg meteor: 0.3991528198808927


 99%|█████████▉| 99/100 [07:07<00:04,  4.26s/it]


avg bleu: 0.5720537260174752

avg meteor: 0.4034694629599928


100%|██████████| 100/100 [07:11<00:00,  4.32s/it]


avg bleu: 0.5775347754359246

avg meteor: 0.4073269330333438


In [29]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,014 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.409100
2,2.379600
3,2.477900
4,2.651300
5,2.444700
6,2.158200
7,2.081500
8,1.839100
9,2.075200
10,1.917400


In [31]:
FastLanguageModel.for_inference(model)

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": "У меня учащенное сердцебиение после физической нагрузки. Это нормально?"}],
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

attention_mask = (inputs != tokenizer.pad_token_id).int().to("cuda")

outputs = model.generate(input_ids=inputs, max_new_tokens = 64, use_cache = True, temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)[0]

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nУ меня учащенное сердцебиение после физической нагрузки. Это нормально?<|im_end|>\n<|im_start|>assistant\nУчащенное сердцебиение может быть связано с различными факторами.  Не все  учашенные  сердцебиения  являются  нормальной.  Обычно,  они  связаны с  физической нагрузкой, стрессом'

In [32]:
bleu = BLEUScore(n_gram=1)
sum_init_bleu_score = 0.
sum_init_meteor_score = 0.

for row in tqdm(eval_ds):
  FastLanguageModel.for_inference(model)
  messages = [row['conversations'][0]]

  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True,
      return_tensors = "pt",
  ).to("cuda")

  outputs = model.generate(input_ids=inputs, max_new_tokens = 64, use_cache = True, temperature = 1.5, min_p = 0.1)
  pred = tokenizer.batch_decode(outputs)[0]
  target = row["text"]
  row["bleu"] = bleu(target, pred)

  metric_bleu = bleu(target, pred)
  sum_init_bleu_score += metric_bleu.item()
  metric_meteor = meteor.compute(predictions=[pred], references=[target])
  sum_init_meteor_score += metric_meteor['meteor']

  print("\navg bleu:", sum_init_bleu_score * 1. / len(eval_ds))
  print("\navg meteor:", sum_init_meteor_score * 1. / len(eval_ds))

  1%|          | 1/100 [00:06<10:59,  6.66s/it]


avg bleu: 0.006688888669013977

avg meteor: 0.003929271507737398


  2%|▏         | 2/100 [00:11<08:49,  5.40s/it]


avg bleu: 0.012807351946830749

avg meteor: 0.00955927498841961


  3%|▎         | 3/100 [00:15<07:38,  4.72s/it]

...


avg bleu: 0.5918604397773742

avg meteor: 0.4184909603105991


 99%|█████████▉| 99/100 [07:04<00:04,  4.31s/it]


avg bleu: 0.5974792516231537

avg meteor: 0.423177405410361


100%|██████████| 100/100 [07:08<00:00,  4.28s/it]


avg bleu: 0.60386814057827

avg meteor: 0.4276240533227338


# Таким образом, результаты следующие

avg bleu: 0.57 -> 0.60

avg meteor: 0.40 -> 0.42

Разница невиликая, но пробный ответ заметно лучше